Bibliotecas:

In [242]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

Exportando os dados:

In [232]:
# Caminhos dos dados e criando datasets para cada conjunto

path = r'D:\Python\dataset\train.csv' # dados de treino
path_teste = r'D:\Python\dataset\test.csv' # dados de teste

dados_teste = pd.read_csv(path_teste)
dados = pd.read_csv(path)

dados_backup = dados.copy() # backup de segurança do dataset

dados.head() # visualizando o dataset


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [233]:
y = dados['SalePrice']
dados.drop('SalePrice', axis=1, inplace=True)

Dados faltantes:

In [234]:
# Visualizando dados faltantes

dados_faltantes = (dados.isnull().sum())
print(dados_faltantes[dados_faltantes > 0])

dados_faltantest = (dados_teste.isnull().sum())

# armazenando os índices onde o número de dados faltantes é relativamente grande
print()
miss = dados_faltantes[dados_faltantes > 600]
print('índices onde o número de dados faltantes é relativamente grande:')
print(miss.index)


LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

índices onde o número de dados faltantes é relativamente grande:
Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


In [235]:
# Criando um novo dataset removendo as colunas com um grande número de dados faltantes

dados = dados.drop(miss.index, axis=1)
dados_teste = dados_teste.drop(miss.index, axis=1)

print(dados.shape, dados_teste.shape)

(1460, 75) (1459, 75)


In [236]:
# Separando os dados em dados numéricos e categóricos

dadosn = dados.select_dtypes(exclude='object')
dadosc = dados.select_dtypes('object')

dadosn_teste = dados_teste.select_dtypes(exclude='object')
dadosc_teste = dados_teste.select_dtypes('object')

dadosc.drop('Condition2', axis=1, inplace=True)
dadosc_teste.drop('Condition2', axis=1, inplace=True)


- Dados faltantes numéricos

In [237]:
# removendo dados faltantes restantes com Simple Imputation

impt = SimpleImputer()
dadosn_impt = pd.DataFrame(impt.fit_transform(dadosn))
dadosn_teste_impt = pd.DataFrame(impt.transform(dadosn_teste))

# Redefinindo as colunas

dadosn_impt.columns = dadosn.columns
dadosn_teste_impt.columns = dadosn_teste.columns

- Dados faltantes categóricos

In [252]:
# utilizando One-Hot Encoding para variáveis com cardinalidade < 5
# utilizando Ordinal Encoding para variáveis com cardinalidade > 5

# selecionando as colunas de variáveis com baixa cardinalidade (bc) e alta cardinalidade (ac)

bc_columns = [col for col in dadosc.columns if dadosc[col].nunique() <= 5]
ac_columns = [col for col in dadosc.columns if dadosc[col].nunique() > 5]

print(f'Baixa cardinalidade: {bc_columns}')
print()
print(f'Alta cardinalidade: {ac_columns}')


Baixa cardinalidade: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']

Alta cardinalidade: ['Neighborhood', 'Condition1', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Functional', 'GarageType', 'SaleType', 'SaleCondition']


In [253]:
# Ordinal Encoding:

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

dadosc_oe = pd.DataFrame(oe.fit_transform(dadosc[ac_columns]))
dadosc_teste_oe = pd.DataFrame(oe.transform(dadosc_teste[ac_columns]))

dadosc_oe.columns = dadosc[ac_columns].columns
dadosc_teste_oe.columns = dadosc_teste[ac_columns].columns


In [254]:
# One-Hot Encoding:

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

dadosc_ohe = pd.DataFrame(ohe.fit_transform(dadosc[bc_columns]))
dadosc_teste_ohe = pd.DataFrame(ohe.transform(dadosc_teste[bc_columns]))

dadosc_ohe.index = dadosc.index
dadosc_teste_ohe.index = dadosc_teste.index

Reunindo os dados tratados:

In [264]:

X = pd.concat([dadosn_impt, dadosc_oe, dadosc_ohe], axis=1)
X_valid = pd.concat([dadosn_teste_impt, dadosc_teste_oe, dadosc_teste_ohe], axis=1)

X.drop(ac_columns, axis=1, inplace=True)
X_valid.drop(ac_columns, axis=1, inplace=True)


In [265]:
# Criando um subgrupo de dados teste/treino para validar e otimizar o modelo

np.random.seed(0)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.7)


Validação cruzada:

In [268]:
resultados = []

k = RepeatedKFold(n_splits=3, n_repeats=10, random_state=0)

for linhas_treino, linhas_teste in k.split(X):

    X_treino, X_teste = X.iloc[linhas_treino], X.iloc[linhas_teste]
    y_treino, y_teste = y.iloc[linhas_treino], y.iloc[linhas_teste]

    modelo = RandomForestRegressor(n_estimators=50, n_jobs=1, random_state=0)

    modelo.fit(X_treino,y_treino)

    p = modelo.predict(X_teste)
    er = abs(y_teste - p)/y_teste*100
    mer = np.mean(er)
    resultados.append(mer)

C:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strin

In [277]:
print(f'Média dos erros relativos: {np.mean(resultados)}')

Média dos erros relativos: 10.659985461645169


Treinando o modelo final:

In [269]:
# Agora com o modelo validado e optimizado, retreinamos com todo o conjunto de dados

modelo = RandomForestRegressor(n_estimators=50, n_jobs=1, random_state=0)
modelo.fit(X,y)

C:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestRegressor(n_estimators=50, n_jobs=1, random_state=0)

Previsão do modelo:

In [275]:
# Previsão para os dados teste:

R = modelo.predict(X_valid)

# Por fim armazenamos o resultado em um arquivo .csv

prev = pd.Series(R, index=dados_teste['Id'], name='SalePrice')
prev.to_csv('House_Prices1.1', header=True)

print(prev)

Id
1461    126459.00
1462    153395.00
1463    178058.80
1464    178567.70
1465    204570.24
          ...    
2915     84964.00
2916     88420.00
2917    151225.74
2918    107771.00
2919    231318.00
Name: SalePrice, Length: 1459, dtype: float64


C:\Users\T-Gamer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
